# Online Course Recommender


---


## Practice Module: Intelligent Reasoning Systems (IRS)

## Data Preparation: Udemy Courses




## File Path & Library Setup & final df setup

In [ ]:
# Load All Necessary Packages

import os
from google.colab import drive

import re
import pandas as pd
import numpy as np

seed = 18

print('Versions of key libraries')
print('-------------------------')
print('pandas:  ', pd.__version__)
print('numpy:   ', np.__version__)

Versions of key libraries
-------------------------
pandas:   1.1.5
numpy:    1.19.5


In [ ]:
# Load Data (Raw Web Scrapped Data - Udemy Courses)
os.chdir('/content')
rawdata = pd.read_csv('v_courses_info_business.csv')

In [ ]:
rawdata.shape

(9986, 14)

In [ ]:
rawdata.head()
df = rawdata.copy()

In [ ]:
final = pd.DataFrame(columns = ['Course Name','Course URL','Categories','Short Description', 'Long Description', 'Difficulty', 'Duration', 'Free Option', 'Rating', 'Original rating', 'Numberofrated', 'Numberofenroll', 'Paid Option', 'Language', 'Subtitle Language', 'Platform', 'Provider', 'Image URL'])
final

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL


## Drop unwanted data

In [ ]:
# remove or modify those incomplete data such as row with uncorrect difficulty, row missing enroll or numbers of rating
difficulty_list = ['All Levels', 'Beginner', 'Intermediate', 'Expert']
df = df[df.level.isin(difficulty_list)]
df.shape

(9984, 14)

In [ ]:
# remove illegal duration row
df = df[df.length.str.contains('total') | df.length.str.contains('question')]
df.shape

(9983, 14)

## Drop non-English courses

In [ ]:
# remove those courses which are not in English
# or no info in language(Udemy no longer provide)
df = df[df.language == 'English']
df.shape

(9892, 14)

In [ ]:
df.head()

,name,description,link,instructors,rating,length,level,bestseller,price,numbersofrating,category,enroll,language,subtitle
0,The Complete SQL Bootcamp 2021: Go from Zero t...,Become an expert at SQL!,https://www.udemy.com/course/the-complete-sql-...,Jose Portilla,4.7,9 total hours,All Levels,Bestseller,$25.99,"(109,210 ratings)","['Business', 'Business Analytics & Intelligenc...","410,023 students",English,"English [Auto], French [Auto]"
1,PMP Exam Prep Seminar - 2021 Exam Content with...,PMP Exam Prep Seminar - Earn 35 PDUs by comple...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,Joseph Phillips,4.6,30 total hours,All Levels,Bestseller,$22.99,"(72,148 ratings)","['Project Management', 'PMBOK']","216,840 students",English,"English [Auto], French [Auto]"
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,Learn Tableau 2020 for data science step by st...,https://www.udemy.com/course/tableau10/,"Kirill Eremenko, Ligency Team",4.6,9 total hours,All Levels,Bestseller,$23.99,"(70,606 ratings)","['Business', 'Business Analytics & Intelligenc...","262,889 students",English,"English [Auto], French [Auto]"
3,Microsoft Power BI - A Complete Introduction [...,"Learn how to use Microsoft's Power BI Tools, i...",https://www.udemy.com/course/powerbi-complete-...,"Manuel Lorenz, Academind by Maximilian Schwarz...",4.6,20 total hours,Beginner,NaN,$19.99,"(50,251 ratings)","['Business Analytics & Intelligence', 'Microso...","180,836 students",English,"English, French [Auto]"
4,Agile Crash Course: Agile Project Management; ...,Get Agile Certified & Learn about the key and ...,https://www.udemy.com/course/agile-crash-course/,Mauricio Rubio - Agile Guru & Founder of Agile...,4.4,2.5 total hours,Beginner,Bestseller,$29.99,"(50,258 ratings)","['Business', 'Project Management', 'Agile']","149,096 students",English,"English [Auto], French [Auto]"


## Fill 0 to those rating is NaN
## Fill None to those subtitle is NaN


In [ ]:
meanval = np.mean(df.rating)
df.rating.fillna(meanval,inplace = True)
df.subtitle.fillna('English(default)',inplace = True)
df.price.fillna('free',inplace = True)
df.numbersofrating.fillna('0',inplace = True)
df.category.fillna('',inplace = True)

## Preprocess Data

### Get Course Name, Short Description, Course URL, Provider Column

In [ ]:
final['Course Name'] = df.name
final['Short Description'] = df.description
final['Course URL'] = df.link
final['Provider'] = df.instructors
final.head()

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,NaN,Become an expert at SQL!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,NaN,PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,NaN,Learn Tableau 2020 for data science step by st...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,NaN,"Learn how to use Microsoft's Power BI Tools, i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,NaN,Get Agile Certified & Learn about the key and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN


### Add Categories

In [ ]:
string = "['asdfa', 'asdf']"
string[1:-1].split(', ')[0][1:-1]

'asdfa'

In [ ]:
def get_cat(strs):
  if strs != '':
    strs = strs[1:-1].split(', ')
    answer = ''
    for item in strs:
      answer = answer+item[1:-1]+','
    return answer[:-1]

In [ ]:
final['Categories'] = df['category'].apply(lambda x: get_cat(x))

### Get Course Difficulty

In [ ]:
# Extract Course Difficulty, converted to numerical category and normalised
def conv_diff(strs):
  if strs == 'All Levels' or strs == 'Beginner':
    diff = 0
  elif strs == 'Intermediate':
    diff = 1
  else:
    diff = 2
  return diff

final['Difficulty'] = df['level'].apply(lambda x: conv_diff(x)) # difficulty in numerical category

final.head()

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN


### Get Course Duration

In [ ]:
string = '0.2 dsfaasdfa 2431 sdfgsdgfsdgsdfg'
re.findall(r'\d+[.]*\d*',string)

['0.2', '2431']

In [ ]:
# Get Course Duration in unit of hours

def duration_conv(strs):
  num = float(re.findall(r'\d+[.]*\d*',strs)[0])
  if 'question' in strs:
    num = num/25
  if 'min' in strs:
    dur = 0
  elif num<10:
    dur = 0
  elif num<50:
    dur = 1
  else:
    dur = 2
  return dur

final['Duration'] = df['length'].apply(lambda x: duration_conv(x))

final.shape

(9892, 18)

In [ ]:
final

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,PMI ACP Agile Certified Practitioner Practice ...,https://www.udemy.com/course/pmi-acp-agile-cer...,"Project Management,PMI-ACP",A practice tests designed to cover all the top...,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ashutosh Deshmukh,NaN
9982,Failure Mode Effect Analysis for Process Risk ...,https://www.udemy.com/course/fmea-for-process-...,"Business,Industry,Failure Mode and Effects Ana...",Include supplier quality audit in FMEA and FME...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Evelyn 7E academy,NaN
9983,Bignners to Pro Course on Financial Modeling a...,https://www.udemy.com/course/bignners-to-pro-c...,"Business,Business Strategy,Financial Modeling",Financial Modeling and Valuation,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harsh Barar,NaN
9984,The Real Estate Invoice Contract Download & Guide,https://www.udemy.com/course/the-real-estate-i...,"Business,Real Estate,Real Estate Investing",Download A Copy Of My PROVEN Real Estate Invoi...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ben Clardy,NaN


### Get free option or not

In [ ]:
def isfree(strs):
  if 'free' in strs:
    f = 1
  else:
    f = 0
  return f

final['Free Option'] = df.price.apply(lambda x:isfree(x))
final.head()

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN


In [ ]:
final.Duration.unique()

array([0, 1, 2])

### Get Course Rating

In [ ]:

string = '(565,433 ratings)'
re.findall(r'\d+[,]*\d*[,]*\d*',string)[0].replace(',','')

'565433'

In [ ]:
# Get Course Rating as normalized number of enrolls

def get_number(strs):
  num = re.findall(r'\d+[,]*\d*[,]*\d*',strs)[0].replace(',','')
  return int(num)

num_enrolls = df.enroll.apply(lambda x:get_number(x))
num_ratings = df.numbersofrating.apply(lambda x:get_number(x))
ratings = df.rating.apply(lambda x:float(x))
maxenroll = max(num_enrolls)
print('the maximum enroll number is:',maxenroll)
final['Original rating'] = ratings
final['Numberofrated'] = num_ratings
final['Numberofenroll'] = num_enrolls
final['Rating'] = ratings/5.0*num_ratings/maxenroll


the maximum enroll number is: 414191


In [ ]:
final.head()

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,0,0,0.247850,4.7,109210,410023,NaN,NaN,NaN,NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,1,0,0.160255,4.6,72148,216840,NaN,NaN,NaN,NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,0,0,0.156830,4.6,70606,262889,NaN,NaN,NaN,NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,1,0,0.111617,4.6,50251,180836,NaN,NaN,NaN,NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,0,0,0.106779,4.4,50258,149096,NaN,NaN,NaN,NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN


### Get Subtitle Language

In [ ]:
string = 'English(defaut)'
string.replace('\xa0',' ').replace(',',' ').split(' ')


['English(defaut)']

In [ ]:
def get_subs(strs):
  lst = strs.replace('\xa0',' ').replace(',',' ').split(' ')
  subs = ''
  for item in lst:
    if item != '' and item != '[Auto]':
      subs = subs+item+','
  return subs[:-1]

get_subs(string)

'English(defaut)'

In [ ]:
# Get course list of subtitle language available

final['Subtitle Language'] = df.subtitle.apply(lambda x:get_subs(x))
final.head()

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,0,0,0.247850,4.7,109210,410023,NaN,NaN,"English,French",NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,1,0,0.160255,4.6,72148,216840,NaN,NaN,"English,French",NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,0,0,0.156830,4.6,70606,262889,NaN,NaN,"English,French",NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,1,0,0.111617,4.6,50251,180836,NaN,NaN,"English,French",NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,0,0,0.106779,4.4,50258,149096,NaN,NaN,"English,French",NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN


### Paid Option

In [ ]:
# Check if free option is available

final['Paid Option'] = df.price.apply(lambda x: 0 if x == 'free' else x)
final.head()

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,0,0,0.247850,4.7,109210,410023,$25.99,NaN,"English,French",NaN,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,1,0,0.160255,4.6,72148,216840,$22.99,NaN,"English,French",NaN,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,0,0,0.156830,4.6,70606,262889,$23.99,NaN,"English,French",NaN,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,1,0,0.111617,4.6,50251,180836,$19.99,NaN,"English,French",NaN,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,0,0,0.106779,4.4,50258,149096,$29.99,NaN,"English,French",NaN,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN


### Adding a Column for platform identifier

In [ ]:
# Add Platform name
# 0 - Edx
# 1 - Udemy

final['Platform'] = [1 for i in range(len(final['Course Name']))]

### Adding a cloumn for language (all English)

In [ ]:
final['Language'] = ['English' for i in range(len(final['Course Name']))]

## Combine data 

In [ ]:
final

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Complete SQL Bootcamp 2021: Go from Zero t...,https://www.udemy.com/course/the-complete-sql-...,"Business,Business Analytics & Intelligence,SQL",Become an expert at SQL!,NaN,0,0,0,0.247850,4.700000,109210,410023,$25.99,English,"English,French",1,Jose Portilla,NaN
1,PMP Exam Prep Seminar - 2021 Exam Content with...,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,"Project Management,PMBOK",PMP Exam Prep Seminar - Earn 35 PDUs by comple...,NaN,0,1,0,0.160255,4.600000,72148,216840,$22.99,English,"English,French",1,Joseph Phillips,NaN
2,Tableau 2020 A-Z: Hands-On Tableau Training fo...,https://www.udemy.com/course/tableau10/,"Business,Business Analytics & Intelligence,Tab...",Learn Tableau 2020 for data science step by st...,NaN,0,0,0,0.156830,4.600000,70606,262889,$23.99,English,"English,French",1,"Kirill Eremenko, Ligency Team",NaN
3,Microsoft Power BI - A Complete Introduction [...,https://www.udemy.com/course/powerbi-complete-...,"Business Analytics & Intelligence,Microsoft Po...","Learn how to use Microsoft's Power BI Tools, i...",NaN,0,1,0,0.111617,4.600000,50251,180836,$19.99,English,"English,French",1,"Manuel Lorenz, Academind by Maximilian Schwarz...",NaN
4,Agile Crash Course: Agile Project Management; ...,https://www.udemy.com/course/agile-crash-course/,"Business,Project Management,Agile",Get Agile Certified & Learn about the key and ...,NaN,0,0,0,0.106779,4.400000,50258,149096,$29.99,English,"English,French",1,Mauricio Rubio - Agile Guru & Founder of Agile...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,PMI ACP Agile Certified Practitioner Practice ...,https://www.udemy.com/course/pmi-acp-agile-cer...,"Project Management,PMI-ACP",A practice tests designed to cover all the top...,NaN,0,1,0,0.000000,4.241297,0,0,$14.99,English,English(default),1,Ashutosh Deshmukh,NaN
9982,Failure Mode Effect Analysis for Process Risk ...,https://www.udemy.com/course/fmea-for-process-...,"Business,Industry,Failure Mode and Effects Ana...",Include supplier quality audit in FMEA and FME...,NaN,0,0,0,0.000000,4.241297,0,0,$14.99,English,English,1,Evelyn 7E academy,NaN
9983,Bignners to Pro Course on Financial Modeling a...,https://www.udemy.com/course/bignners-to-pro-c...,"Business,Business Strategy,Financial Modeling",Financial Modeling and Valuation,NaN,0,0,0,0.000000,4.241297,0,1,$14.99,English,English,1,Harsh Barar,NaN
9984,The Real Estate Invoice Contract Download & Guide,https://www.udemy.com/course/the-real-estate-i...,"Business,Real Estate,Real Estate Investing",Download A Copy Of My PROVEN Real Estate Invoi...,NaN,0,0,0,0.000000,4.241297,0,49,$14.99,English,English,1,Ben Clardy,NaN


## Save Output to file

In [ ]:
filename = 'p_Udemy_business.csv'
final.to_csv('p_datasets/'+filename, index=False, encoding='utf_8_sig')

## Combine all to one

In [ ]:
import os

os.chdir('/content')
dirname = 'p_datasets'
filelist = os.listdir(dirname)
filelist = [file for file in filelist if 'p_' in file]
filelist

['p_Udemy_design.csv',
 'p_Udemy_business.csv',
 'p_Udemy_personalDev.csv',
 'p_Udemy_music.csv',
 'p_Udemy_itNsoftware.csv',
 'p_Udemy_officeProductivity.csv',
 'p_Udemy_lifestyle.csv',
 'p_Udemy_financeNaccounting.csv',
 'p_Udemy_teachNacademic.csv',
 'p_Udemy_marketing.csv',
 'p_Udemy_development.csv',
 'p_Udemy_photoNvideo.csv',
 'p_Udemy_healthNfit.csv']

In [ ]:
os.chdir('p_datasets')
fulldataset = pd.DataFrame()
for file in filelist:
  data = pd.read_csv(file)
  fulldataset = pd.concat([fulldataset,data])


In [ ]:
fulldataset

,Course Name,Course URL,Categories,Short Description,Long Description,Difficulty,Duration,Free Option,Rating,Original rating,Numberofrated,Numberofenroll,Paid Option,Language,Subtitle Language,Platform,Provider,Image URL
0,The Ultimate Drawing Course - Beginner to Adva...,https://www.udemy.com/course/the-ultimate-draw...,"Graphic Design & Illustration,Drawing",Learn the #1 most important building block of ...,NaN,0,1,0,0.200174,4.600000,109305,453470,NT$470,English,"English,French",1,"Jaysen Batchelor, Quinton Ross",NaN
1,Character Art School: Complete Character Drawi...,https://www.udemy.com/course/character-art-sch...,"Design,Other Design,Character Design",Learn How to Draw People and Character Designs...,NaN,0,1,0,0.106542,4.600000,58177,260798,NT$470,English,"English,French",1,Scott Harris,NaN
2,Complete Blender Creator: Learn 3D Modelling f...,https://www.udemy.com/course/blendertutorial/,"Design,3D & Animation,Blender",Use Blender to Create Beautiful 3D models for ...,NaN,0,1,0,0.086477,4.600000,47221,238288,NT$470,English,"English,French",1,"GameDev.tv Team, Rick Davidson, Grant Abbitt",NaN
3,Design Thinking in 3 Steps,https://www.udemy.com/course/designit-design-t...,"Design,User Experience Design,Design Thinking","Understand your audience, envision a creative ...",NaN,0,0,0,0.065123,4.400000,37177,83499,"NT$6,590",English,"English,French",1,"Designit Strategic Design, Alan Cooper",NaN
4,User Experience Design Essentials - Adobe XD U...,https://www.udemy.com/course/ui-ux-web-design-...,"Design,User Experience Design,User Interface","Use XD to get a job in UI Design, User Interfa...",NaN,0,1,0,0.063100,4.600000,34456,136757,NT$470,English,"English,French",1,Daniel Walter Scott,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,Limit Sugar Sweetened Beverages to Not More Th...,https://www.udemy.com/course/limit-sugar-sweet...,"Health & Fitness,Nutrition",Recommendation #21 of 30 for Optimizing Health...,NaN,0,0,1,0.000000,4.446485,0,214,0,English,English,1,"Nicholas Cohen, MD",NaN
5610,Limit Processed Foods to Not More Than One Ser...,https://www.udemy.com/course/limit-processed-f...,"Health & Fitness,Nutrition",Recommendation #22 of 31 for Optimizing Health...,NaN,0,0,1,0.000006,4.446485,1,202,0,English,English,1,"Nicholas Cohen, MD",NaN
5611,how to be an expert in the word of bodybuilding,https://www.udemy.com/course/how-to-be-an-expe...,"Health & Fitness,Fitness,Health",The comprehensive guide: prepares you to be an...,NaN,0,0,1,0.000000,4.446485,0,347,0,English,English,1,Anas Idrissi,NaN
5612,Meditation - The Art of Inner Peace and Happin...,https://www.udemy.com/course/meditation-the-ar...,"Health & Fitness,Meditation",This is Part 5 of Meditation - The Art of Inne...,NaN,0,0,1,0.000000,4.446485,0,222,0,English,English,1,Nima King,NaN


In [ ]:
len(fulldataset['Course Name'].unique())

79286

In [ ]:
fulldataset = fulldataset.drop_duplicates(subset=['Course Name'])

In [ ]:
len(fulldataset)

79286

In [ ]:
fulldataset.to_csv('fulldataset.csv', index=False, encoding='utf_8_sig')